<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/801%20code/T5_Translation_BEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece] datasets
!pip install transformers[torch]
!pip install sacrebleu sentencepiece
!pip install evaluate
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration

raw_datasets = load_dataset("aslg_pc12")
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = T5ForConditionalGeneration.from_pretrained(checkpoint)

raw_datasets = raw_datasets.rename_column("text", "labels")

def tokenize_function(example):
    source = example["gloss"]
    target = example["labels"]

    # Tokenizing source and target without returning tensors and without padding
    tokenized_source = tokenizer(source, truncation=True, max_length=512, return_tensors="pt")
    tokenized_target = tokenizer(target, truncation=True, max_length=512, return_tensors="pt")

    return {
        "input_ids": tokenized_source["input_ids"][0],
        "attention_mask": tokenized_source["attention_mask"][0],
        "labels": tokenized_target["input_ids"][0]
    }

tokenized_datasets = raw_datasets.map(tokenize_function)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)
tokenized_datasets["validation"] = tokenized_datasets.pop("test")

Generating train split:   0%|          | 0/87710 [00:00<?, ? examples/s]

Map:   0%|          | 0/87710 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(output_dir="t5_small_aslg_pc12", push_to_hub=True)
training_args = training_args.set_save(steps=10000)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [11]:
trainer.train()

Step,Training Loss
500,0.477100
1000,0.462300
1500,0.474400
2000,0.452500
2500,0.451700
3000,0.440600
3500,0.441500
4000,0.424500
4500,0.430200
5000,0.405000


TrainOutput(global_step=29604, training_loss=0.38080159929407204, metrics={'train_runtime': 1546.305, 'train_samples_per_second': 153.15, 'train_steps_per_second': 19.145, 'total_flos': 3319457257390080.0, 'train_loss': 0.38080159929407204, 'epoch': 3.0})

In [12]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

'https://huggingface.co/HamdanXI/t5_small_aslg_pc12/tree/main/'